# 01. Marco Teórico Avanzado: Hibridación de Contextos (CRISP-DM Fase 1)

**Autor:** Senior Data Scientist & Principal Academic Investigator  
**Proyecto:** Predicción de inserción laboral (Enfoque Híbrido XGBoost-AFT)  

---

## 1. Visión Científica: El Metamodelo "Chasis + Motor"

La literatura contemporánea (`Saidani 2022`, `Mwita 2024`) sugiere que los modelos puramente curriculares (basados solo en notas y carreras) tienen un techo de rendimiento bajo ($R^2 < 0.4$). Para romper esta barrera, proponemos una arquitectura híbrida:

*   **Chasis (Estructura):** Variables académicas y demográficas (Matriz de Diego). Necesarias pero estáticas.
*   **Motor (Dinámica):** Variables contextuales (Internships, Soft Skills) que aceleran el proceso. `Mwita (2024)` demuestra que las habilidades blandas actúan como multiplicadores de la empleabilidad.

Este notebook valida teóricamente esta aproximación mediante minería de texto sobre el corpus.

In [1]:
import pandas as pd
import glob
import os
import subprocess
import re
import warnings

warnings.filterwarnings('ignore')

# Configuración
LITERATURE_DIR = "../literatura"
BENCHMARK_FILE = "../BrenchmarkCORPUSMEJORADO.ods"

print("Herramientas de análisis cargadas.")

Herramientas de análisis cargadas.


## 2. Validación de Hibridación y Feature Weighting

Buscamos evidencia explícita de autores que ponderan variables ("Feature Weighting") o usan contextos híbridos. 

> **Cita Clave:** `Mwita (2024)` utiliza "weighted means" para cuantificar habilidades técnicas vs blandas.
> **Cita Clave:** `Saidani (2022)` propone "Predicting Student Employability Through the Internship Context".

In [2]:
def search_corpus_for_concepts(concepts, pdf_dir):
    print(f"Buscando conceptos: {list(concepts.keys())} ...")
    pdf_files = glob.glob(os.path.join(pdf_dir, "*.pdf"))
    
    results = []
    for pdf in pdf_files:
        try:
            # Extracción robusta con pdftotext
            res = subprocess.run(['pdftotext', '-layout', pdf, '-'], capture_output=True, text=True, timeout=5)
            text = res.stdout
            
            filename = os.path.basename(pdf)
            for concept, pattern in concepts.items():
                if re.search(pattern, text, re.IGNORECASE):
                     # Extraer snippet
                    m = re.search(r'([^.]*?' + pattern + r'[^.]*?\.)', text, re.IGNORECASE)
                    if m:
                        results.append({'Archivo': filename, 'Concepto': concept, 'Evidencia': m.group(1).strip()})
        except:
            continue
    return pd.DataFrame(results)

# Conceptos Avanzados
conceptos = {
    'Feature Weighting': r'feature weighting|weighted mean|ponderaci',
    'Hybrid Context': r'hybrid approach|internship context|contexto',
    'Soft Skills Impact': r'soft skills|habilidades blandas'
}

df_evidence = search_corpus_for_concepts(conceptos, LITERATURE_DIR)
if not df_evidence.empty:
    display(df_evidence.groupby('Concepto').head(2)) # Muestra 2 ejemplos por concepto

Buscando conceptos: ['Feature Weighting', 'Hybrid Context', 'Soft Skills Impact'] ...


,Archivo,Concepto,Evidencia
0,SkillDiscrepanciesBetween_Brner_2018.pdf,Soft Skills Impact,"Skill discrepancies between research, educatio..."
1,SeeDiscussionsStats_Unknownauthor_2024.pdf,Soft Skills Impact,net/publication/384852714\n\n\n\nProbing the R...
4,TheRoleSoft_Mwita_2024.pdf,Feature Weighting,weighted mean
11,SurvivalNeuralNetworks_Zhang_2020.pdf,Feature Weighting,"Chen L, Wang S (2013) Central clustering of ca..."
17,PredictingStudentEmployability_Saidani_2022.pdf,Hybrid Context,Internship Context
19,EmployabilityAndJob_Casanova_2024.pdf,Hybrid Context,contexto d...


## 3. Justificación: XGBoost-AFT vs Cox PH

La elección de **XGBoost-AFT** no es arbitraria. Se fundamenta en la violación de la asunción de Riesgos Proporcionales (PH) documentada por `Ayaneh et al.` y `Abdulazeez (2024)`. 

*   **Problema de Cox:** Asume que el riesgo relativo es constante en el tiempo ($HR(t) = c$).
*   **Realidad (Ayaneh):** Factores como el género cambian su peso predictivo a lo largo del tiempo de desempleo.
*   **Solución (Barnwal 2021):** AFT modela la aceleración/desaceleración del tiempo, siendo robusto a la no-proporcionalidad.

In [3]:
# Búsqueda específica de la crítica a Cox
cox_critiques = {
    'Cox Violation': r'proportional hazard|PH assumption|violation|fail',
    'AFT Superiority': r'AFT|Accelerated Failure Time|better fit'
}

df_cox = search_corpus_for_concepts(cox_critiques, LITERATURE_DIR)
print("Evidencia contra Cox y a favor de AFT:")
if not df_cox.empty:
    for idx, row in df_cox.head(3).iterrows():
        print(f"[{row['Archivo']}] -> {row['Evidencia']}\n")

Buscando conceptos: ['Cox Violation', 'AFT Superiority'] ...
Evidencia contra Cox y a favor de AFT:
[MachineLearningAlgorithms_Maigur_2024.pdf] -> Two models’ forecasts
are investigated: the proportional and the non-proportional hazard

[MachineLearningAlgorithms_Maigur_2024.pdf] -> (2019), hereaft

[SkillDiscrepanciesBetween_Brner_2018.pdf] -> (i) Students: what jobs might exist in 5–
craft



## 4. Selección de Características: LASSO + RFE

Dado que trabajamos con **Small Data** (cientos de graduados, no millones) y una alta dimensionalidad (Matriz de Diego = 69 dims), el riesgo de *overfitting* es alto. 

Autores como `Suresh (2022)` y `Jayachandran (2024)` recomiendan:
1.  **LASSO (L1 Regularization):** Para forzar coeficientes cero en variables irrelevantes (Sparsity).
2.  **RFE (Recursive Feature Elimination):** Para podar iterativamente variables débiles en el modelo XGBoost.

Esta estrategia será implementada en el **Experimento B**.

## 5. Protocolo Longitudinal (Julio vs Diciembre)

Definimos el algoritmo de actualización de censura para no perder datos:

Sea $T_{obs}$ el tiempo observado y $E$ el indicador de evento ($1=$ Empleo, $0=$ Censurado).

$$ 
T_{final} = \begin{cases} 
\text{Fecha}_{Dic} - \text{Graduación}, & \text{si } E_{Jul}=0 \land E_{Dic}=1 \text{ (Encontró empleo)} \\
\text{Fecha}_{Dic} - \text{Graduación}, & \text{si } E_{Jul}=0 \land E_{Dic}=0 \text{ (Sigue buscando $\rightarrow$ Actualizar Censura)} \\
T_{Jul}, & \text{si } E_{Jul}=1 \text{ (Ya tenía empleo)} 
\end{cases}
$$

Esto maximiza la información temporal disponible.